# 配置

In [1]:
# 系统级
import sys
import os.path
import itertools
import operator
# 第三方
import yaml
import MySQLdb
from openpyxl import load_workbook
from pymongo.mongo_client import MongoClient
# 本地依赖
import config
from loader import ExcelLoader
from model import get_attrs, get_collection
from logger import echo_header
from command import Commander
from util import get_logger, stringify
# 交互
import ipywidgets
from IPython.display import display

paths = []
# Sample
# paths = ['/Users/hqingyi/Ent/百度云同步盘/大数据平台建设/2016数据/24.吉林/2014_院校专业']
# paths.append('')

In [2]:
# 吉林数据
dirname = u'/Users/hqingyi/Ent/百度云同步盘/大数据平台建设/2016数据/24.吉林'
# paths.append(os.path.join(dirname, '2013_专业/.yml'))

# 杜肖 - 吉林(大本)_2013_院校专业_理科_本一B(2)本二本三专科.结果
# paths.append(os.path.join(dirname, '2013_专业/吉林(大本)_2013_院校专业_理科_本一B(2)本二本三专科.结果.专业.yml'))
# paths.append(os.path.join(dirname, '2013_专业/吉林(大本)_2013_院校专业_理科_本一B(2)本二本三专科.结果.院校.yml'))

# 彭遥 2013_院校专业_理科_批前批 & 2013_院校专业_文科_提前批本一
#paths.append(os.path.join(dirname, '2013_专业/吉林(大本)_2013_院校专业_理科_批前批(修改版).结果.专业.yml'))
#paths.append(os.path.join(dirname, '2013_专业/吉林(大本)_2013_院校专业_理科_批前批(修改版).结果.院校.yml'))
#paths.append(os.path.join(dirname, '2013_专业/吉林(大本)_2013_院校专业_文科_提前批本一(修改版).结果.专业.yml'))
#paths.append(os.path.join(dirname, '2013_专业/吉林(大本)_2013_院校专业_文科_提前批本一(修改版).结果.院校.yml'))

# 赵旋丞 2013_文科_本二本三(修改版) & 2013_院校专业_文科_提前批本一
#paths.append(os.path.join(dirname, '2013_专业/吉林(大本)_2013_院校专业_文科_本二本三(修改版).结果.专业.yml'))
#paths.append(os.path.join(dirname, '2013_专业/吉林(大本)_2013_院校专业_文科_本二本三(修改版).结果.院校.yml'))


# 2014年数据
paths.append(os.path.join(dirname, u'2014_院校专业'))

# 分档表
#paths.append(os.path.join(dirname,u'吉林(官网)_分档表_2013~2015.yml'))

 # 初始化

In [3]:
# 初始化并加载
commander = Commander(paths)
commander.load_data()
# 原始结果
commander.summary(save=True)

[DEBUG] 开始解析文件:吉林_文理_院校专业_2014.结果.专业.xlsx
[DEBUG] 开始解析Sheet:Sheet1
[DEBUG] 读取[Sheet1]区域:A2:L12191
[DEBUG] 开始解析文件:吉林_文理_院校专业_2014.结果.院校.xlsx
[DEBUG] 开始解析Sheet:Sheet1
[DEBUG] 读取[Sheet1]区域:A2:K2879


{'error': 0, 'error-obj': 0, 'obj': 15064, 'warning': 0}

# 功能列表

In [4]:
buttons = []
# 校验功能
def _validation(e):
    print u'开始校验...'
    commander.do_validation()
    print commander.summary()
    print u'结束校验...'
btn_validation = ipywidgets.Button(description=u'校验规则', icon='fa-rocket')
btn_validation.on_click(_validation)
buttons.append(btn_validation)

# 校验功能
def _print_error(e):
    print u'开始打印错误内容...'
    print commander.summary()
    commander.echo_error() 
    print u'结束打印...'
btn_print_error = ipywidgets.Button(description=u'打印错误内容', icon='fa-rocket')
btn_print_error.on_click(_print_error)
buttons.append(btn_print_error)

# 基础纠错
def _base_correct(e):
    print u'开始基础纠错(原始属性)'
    commander.do_base_correct()
    print commander.summary()
    print u'结束纠错...'
btn_base_correct = ipywidgets.Button(description=u'基础纠错', icon='fa-rocket')
btn_base_correct.on_click(_base_correct)
buttons.append(btn_base_correct)

# 计算扩展属性
def _ext(e):
    print u'开始计算扩展属性'
    commander.do_extend()
    print commander.summary()
    print u'结束计算扩展属性...'
btn_ext = ipywidgets.Button(description=u'计算扩展属性', icon='fa-rocket')
btn_ext.on_click(_ext)
buttons.append(btn_ext)

# 全面纠错
def _correct(e):
    print u'开始全面纠错'
    commander.do_base_correct()
    print commander.summary()
    print u'结束纠错...'
btn_correct = ipywidgets.Button(description=u'全面纠错', icon='fa-rocket')
btn_correct.on_click(_correct)
buttons.append(btn_correct)


# 全局校验
def _validate_group(e):
    print u'开始全局校验'
    commander.do_group_validation()
    print u'结束校验...'
btn_group_validate = ipywidgets.Button(description=u'全局校验', icon='fa-rocket')
btn_group_validate.on_click(_validate_group)
buttons.append(btn_group_validate)

# 显示全局校验结果
def _show_validate_group(e):
    print u'显示全局校验'
    commander.echo_error(loggers=['order', 'duplicate']) 
    print u'结束校验...'
btn_show_group_validate = ipywidgets.Button(description=u'显示全局校验', icon='fa-rocket')
btn_show_group_validate.on_click(_show_validate_group)
buttons.append(btn_show_group_validate)

# 行级校验
def _validate_row(e):
    print u'开始行级校验'
    commander.do_row_validation()
    print u'结束行级校验...'
btn_row_validate = ipywidgets.Button(description=u'行级校验', icon='fa-rocket')
btn_row_validate.on_click(_validate_row)
buttons.append(btn_row_validate)

# 显示行级校验结果
def _show_validate_row(e):
    print u'显示行级校验'
    commander.echo_error(loggers=['validation_row']) 
    print u'结束行级校验...'
btn_show_row_validate = ipywidgets.Button(description=u'显示行级校验', icon='fa-rocket')
btn_show_row_validate.on_click(_show_validate_row)
buttons.append(btn_show_row_validate)

# 行级纠错
def _correct_row(e):
    print u'开始行级纠错'
    commander.do_row_correction()
    print u'结束行级纠错...'
btn_row_correction = ipywidgets.Button(description=u'行级纠错', icon='fa-rocket')
btn_row_correction.on_click(_correct_row)
buttons.append(btn_row_correction)

for btn in buttons:
    btn.width = '150px'
    display(btn)

# 分级校验及智能纠错

In [5]:
# 校验功能
def _one_key_load(e):
    print u'进行初始化后的校验...'
    commander.do_validation()
    print commander.summary()
    print u'针对基础属性执行纠错功能...'
    commander.do_base_correct()
    commander.do_validation()
    print commander.summary()
    print u'计算扩展属性...'
    commander.do_extend()
    commander.do_validation()
    print commander.summary()
    print u'完整再次纠错...'
    commander.do_correct()
    commander.do_validation()
    print commander.summary()
btn_one_key_load = ipywidgets.Button(description=u' I: 基础校验 & 纠错', icon='fa-rocket')
btn_one_key_load.on_click(_one_key_load)
btn_one_key_load.width = '150px'
display(btn_one_key_load)

btn_print_error = ipywidgets.Button(description=u'打印错误内容', icon='fa-rocket')
btn_print_error.on_click(_print_error)
btn_print_error.width = '150px'
display(btn_print_error)

# 行级校验
def _do_validate_row(e):
    print u'开始行级纠错 & 校验并显示结果'
    commander.do_row_correction()
    commander.do_row_validation()
    commander.echo_error(loggers=['validation_row']) 
    print u'结束行级校验...'
btn_validate3 = ipywidgets.Button(description=u' II: 行级校验 & 纠错', icon='fa-rocket')
btn_validate3.on_click(_do_validate_row)
btn_validate3.width = '150px'
display(btn_validate3)

# 全局校验
def _do_validate_group(e):
    print u'开始全局校验并展示结果'
    commander.do_group_validation()
    commander.echo_error(loggers=['order', 'duplicate', 'continuous_unique']) 
    print u'结束校验...'
btn_validate2 = ipywidgets.Button(description=u' III: 全局校验 & 纠错', icon='fa-rocket')
btn_validate2.on_click(_do_validate_group)
btn_validate2.width = '150px'
display(btn_validate2)

# 抽样校验
def _do_sample_group(e):
    print u'开始抽样校验'
    commander.do_sample_validation()
    print u'结束校验...'
btn_validate4 = ipywidgets.Button(description=u' IV: 抽样校验', icon='fa-rocket')
btn_validate4.on_click(_do_sample_group)
btn_validate4.width = '150px'
display(btn_validate4)

# 抽样校验
def _do_count_group(e):
    print u'开始计数校验'
    commander.do_count_validation()
    print u'结束校验...'
btn_validate5 = ipywidgets.Button(description=u' V: 计数校验', icon='fa-rocket')
btn_validate5.on_click(_do_count_group)
btn_validate5.width = '150px'
display(btn_validate5)

# 手工修正数据

In [6]:
# 校验功能
loggers = ['validation']
def _manual_fix(e):
    print u'开始手工校验...'
    def modify(col_item, man_input):
        def inner(event):
            new_value = man_input.value
            commander.manual_set(col_item, new_value)
        return inner
    for meta, error_datas in commander.loop_error_item(loggers):
        
        display(ipywidgets.HTML(value="<h4>"+commander.str_meta(meta)+"</h4>"))
        for row_data in error_datas:
            display(ipywidgets.HTML(value=commander.str_item(row_data, html=True)))
            display(ipywidgets.HTML(value=commander.str_item_error(row_data, html=True)))
            for logger in loggers:
                errors = get_logger(logger, row_data)
                if errors and len(errors)>0:
                    display(ipywidgets.HTML(value=u"<h6>行级错误:"+logger+"</h6>"))
                    for error in errors:
                        display(ipywidgets.HTML(value=error))
            # 暂时只支持手工修正 ’raw'，以便回写
            for col_data in itertools.chain(row_data['raw'], []):
                for logger in loggers:
                    errors = get_logger(logger, col_data)
                    if errors and len(errors)>0:
                        display(ipywidgets.HTML(value=u"<h6>列级错误:"+logger+"</h6>"))
                        for error in errors:
                            display(ipywidgets.HTML(value=error))
                        
                        value = stringify(col_data['value'])
                        man_input = ipywidgets.Text(description=col_data['key'], value=value)
                        man_click = ipywidgets.Button(description=u'修改')
                        man_click.width = '150px'
                        man_click.on_click(modify(col_data, man_input))

                        display(man_input)
                        display(man_click)

btn_start_fix = ipywidgets.Button(description=u' 手工修正数据', icon='fa-rocket')
btn_start_fix.on_click(_manual_fix)
btn_start_fix.width = '150px'
display(btn_start_fix)


# 保存结果

In [7]:
def _save_to_excel(e):
    print u'把修复写回excel表中...'
    commander.do_save_to_excel()
    print u'结束回写'
btn_save2excel = ipywidgets.Button(description=u' 写回Excel', icon='fa-rocket')
btn_save2excel.on_click(_save_to_excel)
btn_save2excel.width = '250px'
display(btn_save2excel)

def _save_to_mysql(e):
    print u'同步到MySQL数据库中...'
    commander.do_save_to_mysql()
    print u'结束同步'
btn_save2mysql = ipywidgets.Button(description=u' 同步到MySQL数据库', icon='fa-rocket')
btn_save2mysql.on_click(_save_to_mysql)
btn_save2mysql.width = '250px'
display(btn_save2mysql)